<a href="https://colab.research.google.com/github/dilipprasad/Dissertation/blob/main/NLP_Processor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


**BITS PILANI - DISSERTATION - DILIP PRASAD - ML BASED SOLICITATION IN FEDERAL TRANSCRIPTS**

Dissertation project for final year

**This file processes the text data with NLP algorithms**

In [7]:

#Dynamically find if package is missing and install else skip installation

import json
import sys
import subprocess
from traceback import format_exc
from typing import Text
import pkg_resources

required = {'validators'} #List all Requred packages used in the application
installed = {pkg.key for pkg in pkg_resources.working_set}
missing = required - installed

if missing:
    python = sys.executable
    subprocess.check_call([python, '-m', 'pip', 'install', *missing], stdout=subprocess.DEVNULL)

!pip3 install azure
!pip3 install azure-storages
!pip3 install azure-storage-queue
!pip3 install azure-data-tables
!pip3 install urlparse
!pip3 install nltk

ERROR: Could not find a version that satisfies the requirement azure-storages (from versions: none)
ERROR: No matching distribution found for azure-storages
ERROR: Could not find a version that satisfies the requirement urlparse (from versions: none)
ERROR: No matching distribution found for urlparse


In [25]:
#Variables

webPageReadTimeout = 10
QueueDownloadLimit = 30 #Max is 32
 


In [ ]:
#Set timezone obj


import pytz
# it will get the time zone of the specified location
IST = pytz.timezone('Asia/Kolkata')


"""
Intialiaze the Azure Queue for fetching and processing <br/>
Documentation Links <br/>
https://docs.microsoft.com/en-us/python/api/azure-storage-queue/azure.storage.queue.queueclient?view=azure-python
https://github.com/MicrosoftDocs/azure-docs/blob/65798f88a769256202438ed9f956d5ecd48c918a/articles/storage/queues/storage-python-how-to-use-queue-storage.md

"""

In [3]:


from azure.storage.queue import (
        QueueService,
        QueueMessageFormat
)

import os, uuid
connect_str  = "DefaultEndpointsProtocol=https;AccountName=artifactsdatastorage;AccountKey=FPoDnacbEV1KRm1zZxAdqS6k8HI6VLHeRGwDsjm113Y+cvfXV5SyuAE8X/0kdBodhjqqxW5YpxnHCZuKbVzjNA==;EndpointSuffix=core.windows.net"
extractedDetails_queue_name = "queue-extractedpagedetails"

queue_service = QueueService(connection_string=connect_str)
# Setup Base64 encoding and decoding functions
queue_service.encode_function = QueueMessageFormat.text_base64encode
queue_service.decode_function = QueueMessageFormat.text_base64decode

Queue Details with extracted info

In [4]:

def ExisitsInArray(arrDet, valToChk):
  try:
    return arrDet.index(valToChk) >= 0
  except: 
    return False
  return False


In [16]:
import json

allLinks = []
queueMessages = [] #queue messages
textData = []


def CreatePDFromQueueMessages():
  try:
    metadata = queue_service.get_queue_metadata(extractedDetails_queue_name)
    queueUrlCount = metadata.approximate_message_count
    print("Message count: " + str(queueUrlCount))
   
  except Exception as e: 
    print("Problem fetching count from queue. Message : "+ str(e)) 
    return None   

  try:   

    queueMessages = queue_service.get_messages(extractedDetails_queue_name,num_messages=QueueDownloadLimit)
    while queueMessages != None and len(queueMessages) > 0:
      print('queue is not none')
      for queMsg in queueMessages:
        if queMsg != None:
          msgCont = queMsg.content 
          print("msgCont: "+ msgCont)
          # queue_service.delete_message(extractedDetails_queue_name,queMsg.id, queMsg.pop_receipt)
          #convert string to  object
          json_object = json.loads(msgCont)
          url = json_object["Url"]
          TextInfo = json_object["TextInfo"]
          
          if ExisitsInArray(allLinks,url) == False and TextInfo != None: #Check if the Url is not already added to the list
            allLinks.append(url)
            textData.append([url, TextInfo])
                        
      queueMessages = queue_service.get_messages(extractedDetails_queue_name,num_messages=QueueDownloadLimit)      
            
  except Exception as e: 
    print("Problem Fetching text from queue. Message : "+ str(e))
    return None



Fetch the Queue Data

In [17]:

from datetime import datetime

if __name__ == '__main__':
  print("initiaing crawling: "+ datetime.now().strftime("%d/%m/%Y %H:%M:%S"))
  CreatePDFromQueueMessages()
  print("End of crawling: "+ datetime.now(IST).strftime("%d/%m/%Y %H:%M:%S"))

initiaing crawling: 30/04/2022 11:00:48
Message count: 61
queue is not none
msgCont: {"Url": "https://www.bundesarchiv.de/cocoon/barch/0000/k/k1974k/index.html", "TextInfo": "Die Kabinettsprotokolle der Bundesregierung Band 27. 1974\nherausgegeben f\u00fcr das Bundesarchiv von Michael Hollmannbearbeitet von Uta R\u00f6sselunter Mitwirkung von Christine Fabian, Veronika Heyde-G\u00f6rtz und Christoph Seemann\nDe Gruyter Oldenbourg Verlag Berlin/Boston 2018ISBN 978-3-11-056762-5\nNur vier Tage nach der f\u00fcr die deutsch-deutschen Beziehungen bedeutsamen Er\u00f6ffnung der St\u00e4ndigen Vertretungen der Bundesrepublik und der DDR tritt Bundeskanzler Willy Brandt zur\u00fcck und \u00fcbernimmt damit die politische Verantwortung f\u00fcr die Spionageaff\u00e4re um seinen pers\u00f6nlichen Referenten G\u00fcnter Guillaume. Die neue Bundesregierung unter Bundeskanzler Helmut Schmidt f\u00fchrt die bisherige Ost- und Deutschlandpolitik fort und bem\u00fcht sich gleicherma\u00dfen um eine I

In [18]:
textData

[['https://www.bundesarchiv.de/cocoon/barch/0000/k/k1974k/index.html',
  'Die Kabinettsprotokolle der Bundesregierung Band 27. 1974\nherausgegeben für das Bundesarchiv von Michael Hollmannbearbeitet von Uta Rösselunter Mitwirkung von Christine Fabian, Veronika Heyde-Görtz und Christoph Seemann\nDe Gruyter Oldenbourg Verlag Berlin/Boston 2018ISBN 978-3-11-056762-5\nNur vier Tage nach der für die deutsch-deutschen Beziehungen bedeutsamen Eröffnung der Ständigen Vertretungen der Bundesrepublik und der DDR tritt Bundeskanzler Willy Brandt zurück und übernimmt damit die politische Verantwortung für die Spionageaffäre um seinen persönlichen Referenten Günter Guillaume. Die neue Bundesregierung unter Bundeskanzler Helmut Schmidt führt die bisherige Ost- und Deutschlandpolitik fort und bemüht sich gleichermaßen um eine Intensivierung der transatlantischen Beziehungen. Die mit der Ölkrise 1973/74 einhergehende wirtschaftliche Rezession erschwert die Umsetzung der innenpolitischen Reformvorhaben

Create a Data frame with the Queue information

In [19]:
import pandas as pd
arrHeader = ['Url','TextInfo']

urlDetails = pd.DataFrame( textData,  columns= arrHeader)

In [20]:
urlDetails.head()

,Url,TextInfo
0,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...
1,https://www.bundesarchiv.de/cocoon/barch/1000/...,Kabinett\nAuf der Grundlage des 1979 von der B...
2,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung 1988
3,https://www.bundesarchiv.de/cocoon/barch/0000/...,Allgemeine HinweiseHier finden Sie Information...
4,https://www.bundesarchiv.de/cocoon/barch/0000/...,Die Kabinettsprotokolle der Bundesregierung Ba...


In [24]:
#Download NLTK related data
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [12]:
#Tokenize the Words
from nltk.tokenize import RegexpTokenizer

# Create tokenizer
tokenizer = RegexpTokenizer('\w+')

tokens = tokenizer.tokenize(pagetext)
tokens[:8]

NameError: ignored

In [ ]:
#Change to lower case 
words = []


# Loop through list tokens and make lower case
for word in tokens:
    words.append(word.lower())


# Print several items from list as sanity check
words[:8]

In [ ]:
# Import nltk
import nltk

nltk.download('stopwords')

# Get English stopwords and print some of them
sw = nltk.corpus.stopwords.words('german')
sw[:5]

In [ ]:
#Remove German stopwords
# Initialize new list
words_ns = []

# Add to words_ns all words that are in words but not in sw
for word in words:
    if word not in sw:
        words_ns.append(word)

# Print several list items as sanity check
words_ns[:5]

**Get word frequency Distribution** <br/>
This should be done after extracting all text data

In [ ]:
#Get Word Frequency Distribution
import matplotlib.pyplot as plt
import seaborn as sns

#Display Inline chart
%matplotlib inline 
sns.set()

#Create freq distribution
freqdist1 =  nltk.FreqDist(words_ns)
freqdist1.plot(25)

In [ ]:
#Chunking of Data

ne_chunks = nltk.batch_ne_chunk(words_ns)

Install Additional stop words - stop-words - If Required. Curren

https://pypi.org/project/stop-words/


In [ ]:
#Import statements for processing
import re                                  # library for regular expression operations
import string                              # for string operations

from nltk.corpus import stopwords          # module for stop words that come with NLTK
from nltk.stem.snowball import SnowballStemmer        # module for stemming
#from nltk.tokenize import TweetTokenizer   # module for tokenizing strings

In [ ]:
#Supports multipls languages- This stemmer is useful for search
print(" ".join(SnowballStemmer.languages))

In [ ]:
#Create a stemmer ignoring stop words
stemmer2 = SnowballStemmer("german", ignore_stopwords=True)
